This is a variation of the Shift operator where the jobs are shifted using the complex TSP algorithm within the OSRM routing engine.

The output of test is printed out at the end of the notebook

In [1]:
#Importing necessary libraries
import re
import time
import warnings
import numpy as np
import pandas as pd
import math
import requests
import json
import geocoder as gc
import sys
import os
from geopy import distance 
from geopy.geocoders import Nominatim

#Inputs
st = time.time()
sst = time.process_time()
E = ["DY11 5SW","WV4 6ED","B79 7PB","B90 8AT","B46 1AN","B98 9EY","B96 6BD","WV7 3BW","CV9 1LQ"]
J = "B16 9HN"

#TSP function from OSRM
def osrm_tsp(jobs): 
    lat = []
    long = []
    for i in range(len(jobs)):
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.geocode(jobs[i])
        lat.append(location.latitude)
        long.append(location.longitude)
    json_data = []
    lurl = ""
    for i in range(len(jobs)):
        lurl += f'{long[i]},{lat[i]};'
    url = 'http://127.0.0.1:5000/trip/v1/driving/'
    url += lurl[:-1]
    url += '?source=any&destination=any'
    r = requests.get(url)
    json_data.append(r.json())
    for i in range(len(json_data[0]['waypoints'])):
        if(i==0):
            continue
        json_data[0]['waypoints'][i].update({"JID":jobs[i]})
    json_data[0]['waypoints'].sort(key = lambda x:x["waypoint_index"])
    schedule = []
    for i in json_data[0]['waypoints']:
        if('JID' in i):
            schedule.append(i['JID'])
    outputs = {
        "Duration": json_data[0]['trips'][0]['duration'],
        "Schedule": schedule
    }
    return outputs

#Function to calculate time of schedule
def calculate_time_tsp(arr):
    maps = osrm_tsp(arr)
    travel_time = maps["Duration"]
    if (travel_time == -1):
        print("Route not Possible")
        return -1
    travel_time = travel_time/60
    maps["Duration"] = travel_time
    return maps


print("Intial Order: \n")
E.append(J)
print(E)
print("\n")
x = calculate_time_tsp(E)
x["Schedule"].append("DY11 5SW")
x["Schedule"].insert(0,"DY11 5SW")
print("Best Schedule:")
print(x["Schedule"])
print("\nTotal Time Required for Schedule: {} minutes".format(x["Duration"]))
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
eet = time.process_time()
res = eet - sst
print('CPU Execution time:', res, 'seconds')

Intial Order: 

['DY11 5SW', 'WV4 6ED', 'B79 7PB', 'B90 8AT', 'B46 1AN', 'B98 9EY', 'B96 6BD', 'WV7 3BW', 'CV9 1LQ', 'B16 9HN']


Best Schedule:
['DY11 5SW', 'WV4 6ED', 'WV7 3BW', 'B16 9HN', 'CV9 1LQ', 'B79 7PB', 'B46 1AN', 'B90 8AT', 'B98 9EY', 'B96 6BD', 'DY11 5SW']

Total Time Required for Schedule: 227.965 minutes
Execution time: 4.861840486526489 seconds
CPU Execution time: 0.28125 seconds
